In [0]:
from pyspark.sql import SparkSession, functions as f

#Reading Hospital A departments data 
df_hosa=spark.read.parquet("/mnt/bronze/hosipitala/departments")

#Reading Hospital B departments data 
df_hosb=spark.read.parquet("/mnt/bronze/hosipitalb/departments")

#union two departments dataframes
df_merged = df_hosa.unionByName(df_hosb)

# Create the dept_id column and rename deptid to src_dept_id
df_merged = df_merged.withColumn("SRC_Dept_id", f.col("deptid")) \
                     .withColumn("Dept_id", f.concat(f.col("deptid"),f.lit('-'), f.col("datasource"))) \
                     .drop("deptid")

df_merged.createOrReplaceTempView("departments")

In [0]:
%sql
CREATE TABLE IF NOT EXISTS silver.departments (
  Dept_Id STRING,
  SRC_Dept_Id STRING,
  Name STRING,
  datasource STRING,
  is_quarantined BOOLEAN
)
USING DELTA;


In [0]:
%sql
INSERT INTO silver.departments
SELECT 
  Dept_Id,
  SRC_Dept_Id,
  Name,
  datasource,
  CASE 
    WHEN SRC_Dept_Id IS NULL OR Name IS NULL THEN TRUE
    ELSE FALSE
  END AS is_quarantined
FROM departments;


num_affected_rows,num_inserted_rows
40,40
